In [24]:
from elasticsearch import Elasticsearch
PRODUCT_INDEX = 'products'

# Query all the indices that aren't system indices in the elasticsearch
def get_useful_indices():
    es = Elasticsearch()
    filtered_idx = filter(lambda x : '.' not in x, [f['index'] for f in es.cat.indices(format='json')])
    indices_names = [idx for idx in filtered_idx]
    return indices_names

# Get the product in the 'products' index by name
def get_product_by_name(name):
    es = Elasticsearch()
    query_body = {
      "query": {
          "match": {
              "product": name
          }
      }
    }
    possible_products = []
    query = es.search(index=PRODUCT_INDEX, body=query_body)['hits']['hits']
    
    for p in query:
        possible_products.append(p['_source'])
    
    return possible_products

# Get the products names and id's from the 'products' index
def get_products():
    result = []
    es = Elasticsearch()
    query = es.search(index=PRODUCT_INDEX)['hits']['hits']
    
    for p in query:
        p_id = p['_id']
        p_name = p['_source']['product']
        result.append((p_name, p_id))
    return result

In [28]:
get_products()[0]

('Celular Xiaomi Poco X3 6GB/128GB NFC - Shadow Grey', 'UNWdg3gBUC8jSKsEtHo_')